In [44]:
import pandas as pd
import glob
from pathlib import Path

In [45]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None) 

In [46]:
working_dir = '/Users/jahrios/Documents/Stanford/PoldrackLab/Projects/SharedControl/Data/raw'

In [70]:
subjects = glob.glob('/Users/jahrios/Documents/Stanford/PoldrackLab/Projects/SharedControl/Data/raw/sub-*/')

In [57]:
subjects = [Path(subject.rstrip('/')).name[4:] for subject in subjects]

In [67]:
files = glob.glob(f'{working_dir}/sub-*/surveys/*_ai_survey.json')

In [68]:
files

['/Users/jahrios/Documents/Stanford/PoldrackLab/Projects/SharedControl/Data/raw/sub-s003/surveys/s003_ai_survey.json',
 '/Users/jahrios/Documents/Stanford/PoldrackLab/Projects/SharedControl/Data/raw/sub-s002/surveys/s002_ai_survey.json',
 '/Users/jahrios/Documents/Stanford/PoldrackLab/Projects/SharedControl/Data/raw/sub-s001/surveys/s001_ai_survey.json']

In [73]:
Path(files[0].rstrip('/')).name[:4]

's003'

In [17]:
def correct_reverse_coding(row):
    if row['name'] == 'ai_survey_5_options':
        return 6 - row['value']
    else:
        return row['value']
    
def map_value_to_option(row):
    options_list = row['options'].split('|')
    selected_option = options_list[row['value'] - 1].strip() if 0 < row['value'] <= len(options_list) else 'Invalid Value'
    return f"{row['text']} - Selected Option: {selected_option}"


In [86]:
survey_results = {}
for file in files:
    subject = Path(file.rstrip('/')).name[:4]
    
    df = pd.read_json(file)
    df['corrected_value'] = df.apply(correct_reverse_coding, axis=1)
    df['results'] = df.apply(map_value_to_option, axis=1)
    
    result = df['corrected_value'].mean()
    
    survey_results[subject] = {'data': df, 'result': result}

In [87]:
for subject in survey_results.keys():
    print(f"{subject}: {survey_results[subject]['result']}")

s003: 3.5714285714285716
s002: 2.857142857142857
s001: 3.4285714285714284
